##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migration Examples: Canned Estimators

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/canned_estimators"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/canned_estimators.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/canned_estimators.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Canned (or Premade) Estimators have traditionally been used in TensorFlow 1 as quick and easy ways to train models for a variety of typical use cases. TensorFlow 2 provides straightforward approximate substitutes for a number of them by way of Keras models. For those canned estimators that do not have built-in TF2 substitutes, you can still build your own replacement fairly easily.

This guide walks through a few examples of direct equivalents and custom substitutions to demonstrate how TF1's `tf.estimator`-derived models can be migrated to TF2 with Keras.

Namely, this guide includes examples for migrating:
*   `tf.estimator.LinearEstimator`/`Classifier`/`Regressor` -> LinearModel (Keras model defined in this guide)
*   `tf.estimator.DNNEstimator`/`Classifier`/`Regressor` -> Custom Keras DNN Model
*   `tf.estimator.DNNLinearCombinedEstimator`/`Classifier`/`Regressor` -> WideDeepModel  (Keras model defined in this guide)
*   `tf.estimator.BoostedTreesEstimator`/`Classifier`/`Regressor` -> `tfdf.keras.GradientBoostedTreesModel`, `tfdf.keras.RandomForestModel`

A common precursor to the training of a model is feature preprocessing, which is done for TF1 Estimator models with `tf.feature_column`. For more information on feature preprocessing in TF2, see [this guide on migrating feature columns](migrating_feature_columns.ipynb).


## Setup

Start with a couple of necessary TensorFlow imports,

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow_decision_forests as tfdf

prepare some simple data for demonstration from the standard Titanic dataset,

In [ ]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [ ]:
## DATA SETUP FOR ESTIMATORS (TENSORFLOW 1)
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
  feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
  feature_columns.append(feat_col)

and create a method to instantiate a simplistic sample optimizer to use with our various TF1 Estimator and TF2 Keras models.

In [ ]:
def create_sample_optimizer(tf_version):
  if tf_version == 'tf1':
    optimizer = lambda: tf.keras.optimizers.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf1.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.9))
  elif tf_version == 'tf2':
    optimizer = tf.keras.optimizers.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
  return optimizer

## Example 1: Migrating from LinearEstimator

### TF1: Using LinearEstimator

In TF1, you can use `tf.estimator.LinearEstimator` to create a baseline linear model for regression and classification problems.

In [ ]:
linear_estimator = tf.estimator.LinearEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
linear_estimator.train(input_fn=_input_fn, steps=100)
linear_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TF2: Using Keras LinearModel

In TF2, you can create an instance of the Keras `LinearModel` implementation below to substitute for one generated by `tf.estimator.LinearEstimator`, with similar levels of user-specified customization (for example, the ability to customize a chosen model optimizer).

In [ ]:
class LinearModel(keras.engine.training.Model):
  r"""Linear Model for regression and classification problems.

  This model approximates the following function:
  $$y = \beta + \sum_{i=1}^{N} w_{i} * x_{i}$$
  where $$\beta$$ is the bias and $$w_{i}$$ is the weight for each feature.

  Example:

  ```python
  model = LinearModel()
  model.compile(optimizer='sgd', loss='mse')
  model.fit(x, y, epochs=epochs)
  ```

  This model accepts sparse float inputs as well:

  Example:
  ```python
  model = LinearModel()
  opt = tf.keras.optimizers.Adam()
  loss_fn = tf.keras.losses.MeanSquaredError()
  with tf.GradientTape() as tape:
    output = model(sparse_input)
    loss = tf.reduce_mean(loss_fn(target, output))
  grads = tape.gradient(loss, model.weights)
  opt.apply_gradients(zip(grads, model.weights))
  ```

  """

  def __init__(self,
               units=1,
               activation=None,
               use_bias=True,
               kernel_initializer='zeros',
               bias_initializer='zeros',
               kernel_regularizer=None,
               bias_regularizer=None,
               **kwargs):
    """Create a Linear Model.

    Args:
      units: Positive integer, output dimension without the batch size.
      activation: Activation function to use.
        If you don't specify anything, no activation is applied.
      use_bias: whether to calculate the bias/intercept for this model. If set
        to False, no bias/intercept will be used in calculations, e.g., the data
        is already centered.
      kernel_initializer: Initializer for the `kernel` weights matrices.
      bias_initializer: Initializer for the bias vector.
      kernel_regularizer: regularizer for kernel vectors.
      bias_regularizer: regularizer for bias vector.
      **kwargs: The keyword arguments that are passed on to BaseLayer.__init__.
    """

    self.units = units
    self.activation = keras.activations.get(activation)
    self.use_bias = use_bias
    self.kernel_initializer = keras.initializers.get(kernel_initializer)
    self.bias_initializer = keras.initializers.get(bias_initializer)
    self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
    self.bias_regularizer = keras.regularizers.get(bias_regularizer)
    super(LinearModel, self).__init__(**kwargs)
    keras.engine.base_layer.keras_premade_model_gauge.get_cell('Linear').set(True)

  def build(self, input_shape):
    if isinstance(input_shape, dict):
      names = sorted(list(input_shape.keys()))
      self.input_specs = []
      self.dense_layers = []
      for name in names:
        shape = input_shape[name]
        layer = keras.layers.core.Dense(
            units=self.units,
            use_bias=False,
            kernel_regularizer=self.kernel_regularizer,
            name=name)
        self.input_specs.append(
            keras.engine.input_spec.InputSpec(shape=shape, name=name))
        self.dense_layers.append(layer)
    elif isinstance(input_shape, (tuple, list)) and all(
        isinstance(shape, tf.TensorShape) for shape in input_shape):
      self.dense_layers = []
      for shape in input_shape:
        layer = keras.layers.core.Dense(
            units=self.units,
            use_bias=False,
            kernel_initializer=self.kernel_initializer,
            kernel_regularizer=self.kernel_regularizer)
        layer.build(shape)
        self.dense_layers.append(layer)
    else:
      # input_shape can be a single TensorShape or a tuple of ints.
      layer = keras.layers.core.Dense(
          units=self.units,
          use_bias=False,
          kernel_initializer=self.kernel_initializer,
          kernel_regularizer=self.kernel_regularizer)
      layer.build(input_shape)
      self.dense_layers = [layer]

    if self.use_bias:
      self.bias = self.add_weight(
          'bias',
          shape=self.units,
          initializer=self.bias_initializer,
          regularizer=self.bias_regularizer,
          dtype=self.dtype,
          trainable=True)
    else:
      self.bias = None
    self.built = True

  def call(self, inputs):
    result = None
    if isinstance(inputs, dict):
      names = [layer.name for layer in self.dense_layers]
      different_keys = set(names) - set(inputs.keys())
      if different_keys:
        raise ValueError(
            'The input dictionary does not match '
            'the structure expected by the model.'
            '\n\tExpected keys: {}'
            '\n\tReceived keys: {}'
            '\n\tMissing keys: {}'.format(set(names), set(inputs.keys()),
                                          different_keys))
      inputs = [inputs[name] for name in names]
      for inp, layer in zip(inputs, self.dense_layers):
        output = layer(inp)
        if result is None:
          result = output
        else:
          result += output
    elif isinstance(inputs, (tuple, list)):
      for inp, layer in zip(inputs, self.dense_layers):
        output = layer(inp)
        if result is None:
          result = output
        else:
          result += output
    else:
      result = self.dense_layers[0](inputs)

    if self.use_bias:
      result = tf.nn.bias_add(result, self.bias)
    if self.activation is not None:
      return self.activation(result)  # pylint: disable=not-callable
    return result

  def get_config(self):
    config = {
        'units': self.units,
        'activation': keras.activations.serialize(self.activation),
        'use_bias': self.use_bias,
        'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
        'bias_initializer': keras.initializers.serialize(self.bias_initializer),
        'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
        'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
    }
    base_config = keras.engine.base_layer.Layer.get_config(self)
    return dict(list(base_config.items()) + list(config.items()))

  @classmethod
  def from_config(cls, config, custom_objects=None):
    del custom_objects
    return cls(**config)

In [ ]:
linear_model = LinearModel()
linear_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10)
linear_model.evaluate(x_eval, y_eval, return_dict=True)

## Example 2: Migrating from DNNEstimator

### TF1: Using DNNEstimator

In TF1, you can use `tf.estimator.DNNEstimator` to create a baseline DNN model for regression and classification problems.

In [ ]:
dnn_estimator = tf.estimator.DNNEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    hidden_units=[128],
    activation_fn=tf.nn.relu,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
dnn_estimator.train(input_fn=_input_fn, steps=100)
dnn_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TF2: Using Keras to Create a Custom DNN Model

In TF2, you can create a custom DNN model to substitute for one generated by `tf.estimator.DNNEstimator`, with similar levels of user-specified customization (for instance, as in the previous example, the ability to customize a chosen model optimizer).

A similar workflow can be used to replace `tf.estimator.experimental.RNNEstimator` with a Keras RNN Model. Keras provides a number of built-in, customizable choices by way of `tf.keras.layers.RNN`, `tf.keras.layers.LSTM`, and `tf.keras.layers.GRU` - see [here](https://www.tensorflow.org/guide/keras/rnn#built-in_rnn_layers_a_simple_example) for more details.

In [ ]:
dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])

dnn_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])

In [ ]:
dnn_model.fit(x_train, y_train, epochs=10)
dnn_model.evaluate(x_eval, y_eval, return_dict=True)

## Example 3: Migrating from DNNLinearCombinedEstimator

### TF1: Using DNNLinearCombinedEstimator

In TF1, you can use `tf.estimator.DNNLinearCombinedEstimator` to create a baseline combined model for regression and classification problems with customization capacity for both its linear and DNN components.

In [ ]:
optimizer = create_sample_optimizer('tf1')

combined_estimator = tf.estimator.DNNLinearCombinedEstimator(
    head=tf.estimator.BinaryClassHead(),
    # Wide settings
    linear_feature_columns=feature_columns,
    linear_optimizer=optimizer,
    # Deep settings
    dnn_feature_columns=feature_columns,
    dnn_hidden_units=[128],
    dnn_optimizer=optimizer)

In [ ]:
combined_estimator.train(input_fn=_input_fn, steps=100)
combined_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TF2: Using Keras WideDeepModel

In TF2, you can create an instance of the Keras `WideDeepModel` implementation below to substitute for one generated by `tf.estimator.DNNLinearCombinedEstimator`, with similar levels of user-specified customization (for instance, as in the previous example, the ability to customize a chosen model optimizer).

This `WideDeepModel` is constructed on the basis of a constituent `LinearModel` and a custom DNN Model, both of which are created in the preceding two examples. A custom linear model can also be used in place of the built-in Keras `LinearModel` if desired.

In [ ]:
class WideDeepModel(keras.engine.training.Model):
  r"""Wide & Deep Model for regression and classification problems.

  This model jointly train a linear and a dnn model.

  Example:

  ```python
  linear_model = LinearModel()
  dnn_model = keras.Sequential([keras.layers.Dense(units=64),
                               keras.layers.Dense(units=1)])
  combined_model = WideDeepModel(linear_model, dnn_model)
  combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
  # define dnn_inputs and linear_inputs as separate numpy arrays or
  # a single numpy array if dnn_inputs is same as linear_inputs.
  combined_model.fit([linear_inputs, dnn_inputs], y, epochs)
  # or define a single `tf.data.Dataset` that contains a single tensor or
  # separate tensors for dnn_inputs and linear_inputs.
  dataset = tf.data.Dataset.from_tensors(([linear_inputs, dnn_inputs], y))
  combined_model.fit(dataset, epochs)
  ```

  Both linear and dnn model can be pre-compiled and trained separately
  before jointly training:

  Example:
  ```python
  linear_model = LinearModel()
  linear_model.compile('adagrad', 'mse')
  linear_model.fit(linear_inputs, y, epochs)
  dnn_model = keras.Sequential([keras.layers.Dense(units=1)])
  dnn_model.compile('rmsprop', 'mse')
  dnn_model.fit(dnn_inputs, y, epochs)
  combined_model = WideDeepModel(linear_model, dnn_model)
  combined_model.compile(optimizer=['sgd', 'adam'], 'mse', ['mse'])
  combined_model.fit([linear_inputs, dnn_inputs], y, epochs)
  ```

  """

  def __init__(self, linear_model, dnn_model, activation=None, **kwargs):
    """Create a Wide & Deep Model.

    Args:
      linear_model: a premade LinearModel, its output must match the output of
        the dnn model.
      dnn_model: a `tf.keras.Model`, its output must match the output of the
        linear model.
      activation: Activation function. Set it to None to maintain a linear
        activation.
      **kwargs: The keyword arguments that are passed on to BaseLayer.__init__.
        Allowed keyword arguments include `name`.
    """
    super(WideDeepModel, self).__init__(**kwargs)
    keras.engine.base_layer.keras_premade_model_gauge.get_cell('WideDeep').set(True)
    self.linear_model = linear_model
    self.dnn_model = dnn_model
    self.activation = keras.activations.get(activation)

  def call(self, inputs, training=None):
    if not isinstance(inputs, (tuple, list)) or len(inputs) != 2:
      linear_inputs = dnn_inputs = inputs
    else:
      linear_inputs, dnn_inputs = inputs
    linear_output = self.linear_model(linear_inputs)
    # pylint: disable=protected-access
    if self.dnn_model._expects_training_arg:
      if training is None:
        training = keras.backend.learning_phase()
      dnn_output = self.dnn_model(dnn_inputs, training=training)
    else:
      dnn_output = self.dnn_model(dnn_inputs)
    output = tf.nest.map_structure(lambda x, y: (x + y), linear_output,
                                   dnn_output)
    if self.activation:
      return tf.nest.map_structure(self.activation, output)
    return output

  # This does not support gradient scaling and LossScaleOptimizer.
  def train_step(self, data):
    x, y, sample_weight = keras.engine.data_adapter.unpack_x_y_sample_weight(data)

    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)
      loss = self.compiled_loss(
          y, y_pred, sample_weight, regularization_losses=self.losses)
    self.compiled_metrics.update_state(y, y_pred, sample_weight)

    if isinstance(self.optimizer, (list, tuple)):
      linear_vars = self.linear_model.trainable_variables
      dnn_vars = self.dnn_model.trainable_variables
      linear_grads, dnn_grads = tape.gradient(loss, (linear_vars, dnn_vars))

      linear_optimizer = self.optimizer[0]
      dnn_optimizer = self.optimizer[1]
      linear_optimizer.apply_gradients(zip(linear_grads, linear_vars))
      dnn_optimizer.apply_gradients(zip(dnn_grads, dnn_vars))
    else:
      trainable_variables = self.trainable_variables
      grads = tape.gradient(loss, trainable_variables)
      self.optimizer.apply_gradients(zip(grads, trainable_variables))

    return {m.name: m.result() for m in self.metrics}

  def _make_train_function(self):
    # Only needed for graph mode and model_to_estimator.
    has_recompiled = self._recompile_weights_loss_and_weighted_metrics()
    self._check_trainable_weights_consistency()
    # If the loss/weighted metric sub-graphs have been re-compiled then create
    # train function even if one exists already. This is because
    # `_feed_sample_weights` list has been updated on re-compile.
    if getattr(self, 'train_function', None) is None or has_recompiled:
      # Restore the compiled trainable state.
      current_trainable_state = self._get_trainable_state()
      self._set_trainable_state(self._compiled_trainable_state)

      inputs = (
          self._feed_inputs + self._feed_targets + self._feed_sample_weights)
      if not isinstance(keras.backend.symbolic_learning_phase(), int):
        inputs += [keras.backend.symbolic_learning_phase()]

      if isinstance(self.optimizer, (list, tuple)):
        linear_optimizer = self.optimizer[0]
        dnn_optimizer = self.optimizer[1]
      else:
        linear_optimizer = self.optimizer
        dnn_optimizer = self.optimizer

      with keras.backend.get_graph().as_default():
        with keras.backend.name_scope('training'):
          # Training updates
          updates = []
          linear_updates = linear_optimizer.get_updates(
              params=self.linear_model.trainable_weights,  # pylint: disable=protected-access
              loss=self.total_loss)
          updates += linear_updates
          dnn_updates = dnn_optimizer.get_updates(
              params=self.dnn_model.trainable_weights,  # pylint: disable=protected-access
              loss=self.total_loss)
          updates += dnn_updates
          # Unconditional updates
          updates += self.get_updates_for(None)
          # Conditional updates relevant to this model
          updates += self.get_updates_for(self.inputs)

        metrics = self._get_training_eval_metrics()
        metrics_tensors = [
            m._call_result for m in metrics if hasattr(m, '_call_result')  # pylint: disable=protected-access
        ]

      with keras.backend.name_scope('training'):
        # Gets loss and metrics. Updates weights at each call.
        fn = keras.backend.function(
            inputs, [self.total_loss] + metrics_tensors,
            updates=updates,
            name='train_function',
            **self._function_kwargs)
        setattr(self, 'train_function', fn)

      # Restore the current trainable state
      self._set_trainable_state(current_trainable_state)

  def get_config(self):
    linear_config = keras.utils.serialize_keras_object(self.linear_model)
    dnn_config = keras.utils.serialize_keras_object(self.dnn_model)
    config = {
        'linear_model': linear_config,
        'dnn_model': dnn_config,
        'activation': keras.activations.serialize(self.activation),
    }
    base_config = keras.engine.base_layer.Layer.get_config(self)
    return dict(list(base_config.items()) + list(config.items()))

  @classmethod
  def from_config(cls, config, custom_objects=None):
    linear_config = config.pop('linear_model')
    linear_model = keras.layers.deserialize(linear_config, custom_objects)
    dnn_config = config.pop('dnn_model')
    dnn_model = keras.layers.deserialize(dnn_config, custom_objects)
    activation = keras.activations.deserialize(
        config.pop('activation', None), custom_objects=custom_objects)
    return cls(
        linear_model=linear_model,
        dnn_model=dnn_model,
        activation=activation,
        **config)

In [ ]:
# Create LinearModel and DNN Model as in Examples 1 and 2
optimizer = create_sample_optimizer('tf2')

linear_model = LinearModel()
linear_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10, verbose=0)

dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])
dnn_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
combined_model = WideDeepModel(linear_model, dnn_model)
combined_model.compile(
    optimizer=[optimizer, optimizer], loss='mse', metrics=['accuracy'])
combined_model.fit([x_train, x_train], y_train, epochs=10)
combined_model.evaluate(x_eval, y_eval, return_dict=True)

## Example 4: Migrating from BoostedTreesEstimator

### TF1: Using BoostedTreesEstimator

In TF1, you can use `tf.estimator.BoostedTreesEstimator` to create a baseline to create a baseline Gradient Boosting model using an ensemble of decision trees for regression and classification problems.

In [ ]:
bt_estimator = tf1.estimator.BoostedTreesEstimator(
    head=tf.estimator.BinaryClassHead(),
    n_batches_per_layer=1,
    max_depth=10,
    n_trees=1000,
    feature_columns=feature_columns)

In [ ]:
bt_estimator.train(input_fn=_input_fn, steps=1000)
bt_estimator.evaluate(input_fn=_eval_input_fn, steps=100)

### TF2: Using TensorFlow Decision Forests

In TF2, the closest pre-packaged substitute for a model generated by `tf.estimator.BoostedTreesEstimator` is one created using `tfdf.keras.GradientBoostedTreesModel`, which creates a sequentially-trained sequence of shallow decision trees, each designed to "learn" from errors made by its predecessors in the sequence.

`GradientBoostedTreesModel` provides more options for customization, allowing for the specification of everything from basic depth constraints to early stopping conditions. See [here](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel#attributes) for more `GradientBoostedTreesModel` attribute details.

In [ ]:
gbt_model = tfdf.keras.GradientBoostedTreesModel(
    task=tfdf.keras.Task.CLASSIFICATION)
gbt_model.compile(metrics=['mse', 'accuracy'])

In [ ]:
train_df, eval_df = x_train.copy(), x_eval.copy()
train_df['survived'], eval_df['survived'] = y_train, y_eval

train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label='survived')
eval_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(eval_df, label='survived')

gbt_model.fit(train_dataset)
gbt_model.evaluate(eval_dataset, return_dict=True)

In TF2, there is also another available TFDF substitute for a model generated by `tf.estimator.BoostedTreesEstimator` - `tfdf.keras.RandomForestModel`. `RandomForestModel` creates a robust, overfitting-resistant learner consisting of a voting population of deep decision trees, each trained on random subsets of the input training dataset.

`RandomForestModel` and `GradientBoostedTreesModel` provide similarly extensive levels of customization. Choosing between them is problem-specific and dependent on your task or application.

See [here](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel#attributes) for more `RandomForestModel` attribute details.

In [ ]:
rf_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.CLASSIFICATION)
rf_model.compile(metrics=['mse', 'accuracy'])

In [ ]:
rf_model.fit(train_dataset)
rf_model.evaluate(eval_dataset, return_dict=True)